In [5]:
import sys
import time  #import time用以設定延遲秒數
import pickle
import requests
from datetime import datetime
from bs4 import BeautifulSoup

In [2]:
def get_date(news_block_node): #找出日期
    date_string = news_block_node.find(class_="news_date").string.split('|')[0][2:-1] #找出新聞發布日
    return(datetime.strptime(date_string, '%Y.%m.%d').strftime('%Y-%m-%d'))
    
def get_title(news_block_node): #找出新聞標題
    return news_block_node.find(class_="news_title").a.string

def get_link(news_block_node):#找出新聞網址
    return news_block_node.find(class_="news_title").a.get('href')

def get_content(link): #找出內文 用UTF-8編碼，以免亂碼
    r = requests.get(link)
    r.encoding = "UTF-8"
    soup = BeautifulSoup(r.text, 'html.parser')
    article_node = soup.find(itemprop='articleBody')
    article = article_node.get_text()
    return article.replace("\n", "")

In [58]:
def get_news_info(each_news): #對每則新聞的Body做設定
    date  = get_date(each_news)
    title = get_title(each_news)
    link  = get_link(each_news)
    content = get_content(link)
    
    info = {'date' : date,
            'title': title,
            'link' : link,
            'content': content}
    return(info)

def get_page_news(page_url): #抓取連結
    r = requests.get(page_url)
    r.encoding = "UTF-8"

    soup = BeautifulSoup(r.text, 'html.parser')
    news_blocks = soup.find_all(class_="news-list-item clearfix ")
    
    news = []
    for each_news in news_blocks:   #迴圈 使用 Try Except 剔除掉Error資料
        try:
            news_info = get_news_info(each_news)
#             print(get_title(each_news))
            time.sleep(2)           #設定二秒延遲
        except:
#             print('-------{}-------'.format())
            pass

        news.append(news_info)
    return(news)


def get_new_talk_news(from_page=1, end_page=270, url="https://newtalk.tw/news/subcategory/2/政治/"): 
    '''
    設定頁數(政治新聞版)
    '''
    print("page_number from {} to {}".format(from_page, end_page-1))
    data = []
    for page_number in range(from_page, end_page):
        print("page_number: {}".format(page_number))
        data = data + get_page_news( url+str(page_number) )
        
    print('done')
    return(data)